In [182]:
import os,sys
import logging, time
from pprint import pprint, pformat
import importlib
from datetime import timedelta, datetime
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as py
from plotly import tools, offline
#import cufflinks as cf
import qgrid

from docs.config import *
from docs.data import *
import app
from app import freqtostr, strtofreq, pertostr
from app.common.timer import Timer
from app.common.utils import utc_datetime as now, utc_dtdate
from app.bnc import *
from app.bnc import analyze, candles, signals, trade, strategy

# Conf
log = logging.getLogger("testing")
pd.set_option("display.max_columns", 25)
pd.set_option("display.width", 2000)
app.set_db(["localhost", "45.79.176.125"][0])
db = app.get_db()


# Init plotly
cred = db.api_keys.find_one({"name":"plotly"})
tools.set_credentials_file(username=cred['plotly_user'], api_key=cred['plotly_api_key'])
offline.init_notebook_mode(connected=True)
#cf.go_offline()

# Generate plot data
freq=300
freq_str='5m'
pair='LTCUSDT'
trade.freq = freq
trade.freq_str = freq_str
trade.init()
dfc = app.bnc.dfc
btc = dfc.loc[pair,freq].tail(288)
df = signals.MACD(btc, 12, 26)

qscale = qgrid.show_grid(
    df,
    show_toolbar=True,
    grid_options={'forceFitColumns': True, 'defaultColumnWidth': 100})

In [183]:
price = go.Scatter(x=df.index,y=df['close'])

macd = go.Scatter(x=df.index, y=df['macd'])
macd_sign = go.Scatter(x=df.index, y=df['macd_sign'])
macd_diff = go.Bar(x=df.index, y=df['macd_diff']) #, filename='basic histogram')

fig = tools.make_subplots(
    rows=2, cols=1,
    specs=[ [{}], [{}] ],
    shared_xaxes=True, shared_yaxes=True,
    vertical_spacing=0.001)
fig.append_trace(price, 1, 1)
fig.append_trace(macd, 2, 1)
fig.append_trace(macd_diff, 2, 1)
fig.append_trace(macd_sign, 2, 1)


fig['layout'].update(title='BTC w/ MACD')

py.iplot(fig)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]



In [184]:
qscale

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defaultColumnWidth': 100, 'rowHeight': 28, 'enableColumnReorder': False, 'enableTextSelectionOnCells': True, 'editable': True, 'autoEdit': False, 'explicitInitialization': True, 'maxVisibleRows': 15, 'minVisibleRows': 8, 'sortable': True, 'filterable': True, 'highlightSelectedCell': False, 'highlightSelectedRow': True}, precision=5, show_toolbar=True)

In [188]:

df2 = df.tail(25)
last = np.float64(df2.tail(1)['macd_diff'])
cut = df2[df2['macd_diff'] > 0].iloc[-1] if last < 0 else df2[df2['macd_diff'] < 0].iloc[-1]
hist = df2.loc[slice(cut.name, df2.iloc[-1].name)].iloc[1:-1]
mean = hist['macd_diff'].describe()['mean']
hist['macd_diff'].describe().round(5).to_dict()


{'25%': 0.06678,
 '50%': 0.07844,
 '75%': 0.10237,
 'count': 3.0,
 'max': 0.12629,
 'mean': 0.08662,
 'min': 0.05513,
 'std': 0.03628}